# Análisis de los resultados de la minería de reglas

In [ ]:
!pip install seaborn

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import ast
import networkx as nx
import seaborn as sns
import numpy as np
import networkx as nx
from matplotlib.patches import Patch

In [ ]:
import re
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, udf, size, split
from pyspark.sql.types import BooleanType
from pyspark.sql.functions import size

# Preparar dataset con diagnósticos como consecuente para análisis

In [ ]:
# Detener sesión anterior
try:
    spark.stop()
except:
    pass


spark = SparkSession.builder \
    .appName("Preparación dataset para entrenamiento del modelo predictivo") \
    .config("spark.master", "local[*]") \
    .config("spark.driver.memory", "24g") \
    .config("spark.executor.memory", "24g") \
    .getOrCreate()

In [ ]:
ruta_rules = "data/resultados/fpgrowth_rules.parquet"

In [ ]:
df_spark = spark.read.parquet(ruta_rules, header=True, inferSchema=True)

In [ ]:
ruta_labitems = "data/mimic4/d_labitems.csv"
df_labitems = pd.read_csv(ruta_labitems)

In [ ]:
df_labitems.head()

Se carga también el datset con las etiquetas bioquímicas para facilitar la interpretación de los items obtenidos en el antecedente. El uso de códigos numéricos optimiza el almacenamiento y procesamiento de los datos, ya que las "label" pueden ser frases largas que incrementan innecesariamente la dimensionalidad del dataset.

In [ ]:
# Filtrar reglas con un diagnóstico como consecuente: Letra + 2 números

# Expresión regular para identificar diagnósticos
diagnosis_pattern = r"^[A-Za-z][0-9]{2}$"

@udf(BooleanType())
def has_diagnosis_udf(consequent_items):
    if isinstance(consequent_items, list):
        return any(re.match(diagnosis_pattern, item) for item in consequent_items)
    return False

# Reglas donde el consecuente es un diagnóstico
df_filtrado = df_spark.filter(has_diagnosis_udf(col("consequent")))


In [ ]:
df_filtrado.show(10)

In [ ]:
# Contar el número de antecedentes
df_filtrado = df_filtrado.withColumn("num_antecedents", size(col("antecedent")))

# Filtrar reglas con 5 o menos antecedentes
df_final = df_filtrado.filter(col("num_antecedents") <= 5)

In [ ]:
df_final.show(10, truncate=False)

In [ ]:
# Mostrar la cantidad de reglas después del filtrado
print(f"Después del filtrado hay {df_final.count()} reglas con diagnósico como consecuente")

In [ ]:
# Guardar en formato Parquet
ruta_salida_parquet = "data/resultados/reglas_diagnostico_filtradas_5antecedent.parquet"

df_final.coalesce(1).write.mode("overwrite").parquet(ruta_salida_parquet)

print(f"Archivo guardado en: {ruta_salida_parquet}")

# Análisis de las reglas generadas

In [ ]:
# Cargar dataset de reglas filtradas: Diagnostico consecuente y menos de 6 antecedentes
df_diagnosticos_reglas = pd.read_parquet(ruta_salida_parquet)

df_diagnosticos_reglas.head()

In [ ]:
df_diagnosticos_reglas.head(50)

In [ ]:
print(df_diagnosticos_reglas.dtypes)

In [ ]:
print(type(df_diagnosticos_reglas["antecedent"].iloc[0]))

In [ ]:
# Convertir las columnas de arrays de NumPy a listas de Python
df_diagnosticos_reglas["antecedent"] = df_diagnosticos_reglas["antecedent"].apply(lambda x: x.tolist() if isinstance(x, np.ndarray) else x)
df_diagnosticos_reglas["consequent"] = df_diagnosticos_reglas["consequent"].apply(lambda x: x.tolist() if isinstance(x, np.ndarray) else x)

print(type(df_diagnosticos_reglas["antecedent"].iloc[0]))
print(type(df_diagnosticos_reglas["consequent"].iloc[0]))

In [ ]:
# Frecuencia diagnósticos en el consecuente
consequent_counts = df_diagnosticos_reglas["consequent"].apply(lambda x: x[0] if len(x) > 0 else None).value_counts()
print(consequent_counts)

In [ ]:
# Contar el número de reglas generadas
num_rules_filtradas = df_diagnosticos_reglas.count()

print(f"El modelo generó un total de {num_rules_filtradas} reglas con diagnostico como consecuente.")

In [ ]:
# Histogramas con la distribución de las métricas que evaluan las reglas de asociación
fig, axes = plt.subplots(1, 3, figsize=(15, 5))

# Soporte
df_diagnosticos_reglas["support"].hist(ax=axes[0], bins=30, color="lightcoral", alpha=0.7, edgecolor="white")
axes[0].set_title("Distribución de Support")

# Confianza
df_diagnosticos_reglas["confidence"].hist(ax=axes[1], bins=30, color="orangered", alpha=0.7, edgecolor="white")
axes[1].set_title("Distribución de Confidence")

# Lift
df_diagnosticos_reglas["lift"].hist(ax=axes[2], bins=30, color="darkred", alpha=0.7, edgecolor="white")
axes[2].set_title("Distribución de Lift")

plt.show()

## Reglas con el lift más alto (mayor de 10)

In [ ]:
# Filtrar reglas con lift mayor de 10
df_filtrado_lift10 = df_diagnosticos_reglas[
    (df_diagnosticos_reglas['lift'] > 10)
]

df_filtrado_lift10.head()

In [ ]:
# Función para extraer los parámetros bioquímicos (items de 5 dígitos)
def extract_bioquimicos(antecedent):
    return [item for item in antecedent if item.isdigit() and len(item) == 5]

df_diagnosticos_reglas["bioquimicos"] = df_diagnosticos_reglas["antecedent"].apply(extract_bioquimicos)

print(df_diagnosticos_reglas["bioquimicos"].head())

In [ ]:
# Expandir la columna 'bioquimicos' para que cada parámetro tenga su propia fila
df_bioquimicos_exploded = df_diagnosticos_reglas.explode("bioquimicos")

# Contar la frecuencia de cada parámetro bioquímico
conteo_bioquimicos = df_bioquimicos_exploded["bioquimicos"].value_counts().reset_index()
conteo_bioquimicos.columns = ["Parametro_Bioquimico", "Frecuencia"]

# Convertir la columna a numérico para hacer merge
conteo_bioquimicos["Parametro_Bioquimico"] = pd.to_numeric(conteo_bioquimicos["Parametro_Bioquimico"], errors="coerce")

In [ ]:
# Añadir las etiquetas a conteo_bioquimicos
conteo_bioquimicos_merged = conteo_bioquimicos.merge(
    df_labitems[['itemid', 'label']],
    left_on="Parametro_Bioquimico",
    right_on="itemid",
    how="left"
)

conteo_bioquimicos_merged = conteo_bioquimicos_merged.drop(columns=['itemid'])

In [ ]:
conteo_bioquimicos_merged.head()

## Reglas con el lift bajo

In [ ]:
# Filtrar reglas con lift entre 2 y 3
df_filtrado2_3 = df_diagnosticos_reglas[
    (df_diagnosticos_reglas['lift'].between(2, 3))
]

df_filtrado2_3.head()

In [ ]:
df_filtrado2_3 = df_filtrado2_3.copy()

# Aplicar la función y expandir la columna de parámetros bioquímicos
df_filtrado2_3["bioquimicos"] = df_filtrado2_3["antecedent"].apply(extract_bioquimicos)
df_bioquimicos_exploded2_3 = df_filtrado2_3.explode("bioquimicos")

# Contar la frecuencia y realizar el merge con df_labitems
conteo_bioquimicos2_3 = df_bioquimicos_exploded2_3["bioquimicos"].value_counts().reset_index()
conteo_bioquimicos2_3.columns = ["Parametro_Bioquimico", "Frecuencia"]

conteo_bioquimicos2_3["Parametro_Bioquimico"] = pd.to_numeric(conteo_bioquimicos2_3["Parametro_Bioquimico"], errors="coerce")
conteo_bioquimicos_merged2_3 = conteo_bioquimicos.merge(df_labitems[['itemid', 'label']],
                                                      left_on="Parametro_Bioquimico",
                                                      right_on="itemid",
                                                      how="left")

conteo_bioquimicos_merged2_3 = conteo_bioquimicos_merged2_3.drop(columns=['itemid'])

conteo_bioquimicos_merged2_3.head()

In [ ]:
# Filtrar reglas con lift entre 1,7 y 2
df_filtrado_menos2 = df_diagnosticos_reglas[
    (df_diagnosticos_reglas['lift'].between(1.7, 2))
]

df_filtrado_menos2.head()

In [ ]:
df_filtrado_menos2 = df_filtrado_menos2.copy()

# Aplicar la función y expandir la columna de parámetros bioquímicos
df_filtrado_menos2["bioquimicos"] = df_filtrado_menos2["antecedent"].apply(extract_bioquimicos)
df_bioquimicos_exploded_menos2 = df_filtrado_menos2.explode("bioquimicos")

# Contar la frecuencia y realizar el merge con df_labitems
conteo_bioquimicos_menos2 = df_bioquimicos_exploded_menos2["bioquimicos"].value_counts().reset_index()
conteo_bioquimicos_menos2.columns = ["Parametro_Bioquimico", "Frecuencia"]

conteo_bioquimicos_menos2["Parametro_Bioquimico"] = pd.to_numeric(conteo_bioquimicos_menos2["Parametro_Bioquimico"], errors="coerce")
conteo_bioquimicos_merged_menos2 = conteo_bioquimicos.merge(df_labitems[['itemid', 'label']],
                                                      left_on="Parametro_Bioquimico",
                                                      right_on="itemid",
                                                      how="left")

conteo_bioquimicos_merged_menos2 = conteo_bioquimicos_merged_menos2.drop(columns=['itemid'])

conteo_bioquimicos_merged_menos2.head()

# Visualización gráfica de las conclusiones

In [ ]:
# Funcion que categoriza los antecedentes en 3 categorías:
#   bioquimicos: parámetros bioquimicos alterados (5 dígitos)
#   diagnosticos: diagnósticos médicos (letra + 2 dígitos)
#   otros: restos de variables sociodemográficas que aparecen en las reglas

def categorizar_antecedentes(antecedent):
    if isinstance(antecedent, list):
        bioquimicos = [item for item in antecedent if item.isdigit() and len(item) == 5]                # Números de 5 dígitos
        diagnosticos = [item for item in antecedent if re.match(r"^[A-Za-z]\d{2}$", item)]              # Diagnósticos (Letra + 2 dígitos)
        otros = [item for item in antecedent if item not in bioquimicos and item not in diagnosticos]   # Otras variables
        return bioquimicos, diagnosticos, otros
    return [], [], []  # Si no es una lista, devolver listas vacías


In [ ]:
df_filtrado_lift10 = df_filtrado_lift10.copy()

# Aplicar función categorizar_antecedentes para lift mayor de 10
df_filtrado_lift10["bioquimicos"], df_filtrado_lift10["diagnosticos"], df_filtrado_lift10["otros"] = zip(
    *df_filtrado_lift10["antecedent"].apply(categorizar_antecedentes)
)

# Crear el grafo
G = nx.DiGraph()

# Iterar sobre las reglas y añadir nodos y aristas
for _, row in df_filtrado_lift10.iterrows():
    # Convertir el consecuente en string si es una lista
    consecuente = row["consequent"][0] if isinstance(row["consequent"], list) and len(row["consequent"]) > 0 else str(row["consequent"])

    bioquimicos = row["bioquimicos"]
    diagnosticos = row["diagnosticos"]
    otros = row["otros"]

    # Añadir nodos con colores diferenciados
    for b in bioquimicos:
        G.add_node(b, color="sandybrown")

    for d in diagnosticos:
        G.add_node(d, color="mistyrose")

    for o in otros:
        G.add_node(o, color="darkgrey")

    # Nodo del consecuente
    G.add_node(consecuente, color="salmon")

    # Crear aristas hacia el consecuente
    for b in bioquimicos:
        G.add_edge(b, consecuente)

    for d in diagnosticos:
        G.add_edge(d, consecuente)

    for o in otros:
        G.add_edge(o, consecuente)

        
# Crear la figura
fig, ax = plt.subplots(figsize=(12, 8))

# Extraer colores
colors = [G.nodes[node].get("color", "lightgray") for node in G.nodes]

# Disposición de nodos
pos = nx.spring_layout(G, k=0.7)

# Dibujar el grafo en el eje `ax`
nx.draw(
    G, pos, ax=ax, with_labels=True, node_color=colors, edge_color="gray",
    node_size=1000, font_size=10, alpha=0.9, width=2
)

# Configurar el título correctamente
ax.set_title("Red de asociaciones entre diagnósticos, alteraciones bioquímicas y otras variables con lift > 10")

# Mostrar la figura
plt.show()

In [ ]:
# Aplicar función categorizar_antecedentes para todos los diagnósticos
df_diagnosticos_reglas["bioquimicos"], df_diagnosticos_reglas["diagnosticos"], df_diagnosticos_reglas["otros"] = zip(
    *df_diagnosticos_reglas["antecedent"].apply(categorizar_antecedentes)
)

# Obtener lista de diagnósticos que aparecen como consecuentes
diagnosticos_consecuente = set(df_diagnosticos_reglas["consequent"].apply(lambda x: x[0] if isinstance(x, list) and len(x) > 0 else str(x)))

# Crear el grafo
G = nx.DiGraph()

# Iterar sobre las reglas y añadir nodos y aristas
for _, row in df_diagnosticos_reglas.iterrows():
    # Convertir el consecuente en string si es una lista
    consecuente = row["consequent"][0] if isinstance(row["consequent"], list) and len(row["consequent"]) > 0 else str(row["consequent"])

    bioquimicos = row["bioquimicos"]
    diagnosticos = row["diagnosticos"]
    otros = row["otros"]

    # Añadir nodos con colores diferenciados
    for b in bioquimicos:
        G.add_node(b, color="sandybrown")

    for d in diagnosticos:
        if d in diagnosticos_consecuente:
            G.add_node(d, color="indianred")
        else:
            G.add_node(d, color="mistyrose")

    for o in otros:
        G.add_node(o, color="darkgrey")

    # Nodo del consecuente
    G.add_node(consecuente, color="salmon")

    # Crear aristas hacia el consecuente
    for b in bioquimicos:
        G.add_edge(b, consecuente)

    for d in diagnosticos:
        G.add_edge(d, consecuente)

    for o in otros:
        G.add_edge(o, consecuente)

# Dibujar el grafo
plt.figure(figsize=(15, 10))
pos = nx.spring_layout(G, k=1)

# Obtener colores de los nodos
colors = [G.nodes[node]["color"] for node in G.nodes]

# Dibujar nodos
nx.draw_networkx_nodes(G, pos, node_color=colors, node_size=700, alpha=0.9)

# Dibujar etiquetas
nx.draw_networkx_labels(G, pos, font_size=8)

# Dibujar aristas
edges = nx.draw_networkx_edges(
    G, pos, edge_color="lightseagreen", alpha=0.6, width=0.8,
    connectionstyle="arc3,rad=0.2")

# Leyenda
legend_patches = [
    Patch(color="salmon", label="Diagnósticos consecuente"),
    Patch(color="mistyrose", label="Diagnósticos antecedente"),
    Patch(color="indianred", label="Diagnósticos en antecedente y consecuente"),
    Patch(color="sandybrown", label="Parámetros bioquímicos"),
    Patch(color="darkgrey", label="Otras variables")
]

plt.legend(handles=legend_patches, loc="upper left", title="Categoría de los nodos", fontsize=9)

plt.title("Asociaciones entre los ítems de las reglas de asociación con consecuente DIAGNÓSTICO")
plt.show()

In [ ]:
df_diagnosticos_reglas_copy = df_diagnosticos_reglas.copy()
df_diagnosticos_reglas_copy["consequent"] = df_diagnosticos_reglas_copy["consequent"].apply(lambda x: x[0] if isinstance(x, list) and len(x) > 0 else str(x))

# Ordenar diagnósticos por lift promedio
mean_lift_per_consequent = df_diagnosticos_reglas_copy.groupby("consequent")["lift"].mean().sort_values()

plt.figure(figsize=(12, 6))
sns.violinplot(
    data=df_diagnosticos_reglas_copy,
    x="consequent",
    y="lift",
    order=mean_lift_per_consequent.index,
    palette="coolwarm",
    inner="quartile"
)

plt.xticks(rotation=90)
plt.xlabel("Diagnóstico Consecuente")
plt.ylabel("Lift")
plt.title("Distribución del Lift por Diagnóstico Consecuente")
plt.grid(True)

plt.show()

El análisis de la distribución del lift por diagnóstico consecuente revela que cada diagnóstico presenta un rango característico de lift, lo que indica que las reglas con el mismo consecuente tienden a agruparse en torno a valores similares.
Diagnósticos como E78, Z79 y Y92 muestran lifts bajos y homogéneos, mientras que otros, como I50 y N18, presentan mayor variabilidad, sugiriendo la existencia de subgrupos con diferentes niveles de asociación.
En contraste, diagnósticos como I13 exhiben un lift consistentemente alto (>10), lo que evidencia asociaciones excepcionalmente fuertes con sus antecedentes.


In [ ]:
plt.figure(figsize=(12, 6))
sns.histplot(data=df_diagnosticos_reglas_copy, x="lift", hue="consequent", bins=20, element="step", stat="count", common_norm=False)
plt.xlabel("Lift")
plt.ylabel("Número de Reglas")
plt.title("Distribución del Lift por Diagnóstico Consecuente")
plt.legend(title="Diagnóstico")
plt.show()